In [1]:
import pandas as pd
import itertools
import warnings
import numpy as np
from sklearn import preprocessing
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import numpy as np
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Flatten
from tensorflow.keras.optimizers import SGD,Adam

from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn import metrics

from sklearn.feature_selection import RFE

## Firstly, we deal with the dataset "1 years before"

## Presenting the data

In [2]:
df=pd.read_csv("1 years before.csv")
df['final'][df['final']=='Bankrupted']=0
df['final'][df['final']=='Non-Bankrupted']=1
df.head()

bankruptcies=df['final'][df['final']==0].count()
rows=df.shape[0]
columns=df.shape[1]
print(f'Our dataset is comprised of {rows} rows and {columns} columns. We have {bankruptcies} bankrupted companies and {rows-bankruptcies} companies that still operate.')

Our dataset is comprised of 145 rows and 25 columns. We have 49 bankrupted companies and 96 companies that still operate.


In [3]:
df

,SIZE-3year,GR-PROFIT-MARGIN-3year,CAPITAL-EMPLOYED-TURNOVER-3year,STOCKHOLDERS-EQUITY-TURNOVER-3year,CAPITAL-EMPLOYED/NET-FIXED-ASSETS-3year,DEBT/EQUITY-3year,EQUITY/CAPITAL-EMPLOYED-3year,WORKING-CAPITAL/TOTAL-ASSETS-3year,AV-COLLECTION-PERIOD-FOR-RECIEVABLES-3year,AV-PAYMENT-PERIOD-3year,...,CAPITAL-EMPLOYED-TURNOVER-2year,STOCKHOLDERS-EQUITY-TURNOVER-2year,CAPITAL-EMPLOYED/NET-FIXED-ASSETS-2year,DEBT/EQUITY-2year,EQUITY/CAPITAL-EMPLOYED-2year,WORKING-CAPITAL/TOTAL-ASSETS-2year,AV-COLLECTION-PERIOD-FOR-RECIEVABLES-2year,AV-PAYMENT-PERIOD-2year,AV-TURNOVER-PERIOD-FOR-INVENTORIES-2year,final
0,17.048357,11.394002,8.553323,9.782489,0.545139,22.955679,0.874350,-0.338667,431.196373,307.433926,...,4.437544,5.073205,1.765993,16.508362,0.874702,0.010900,698.486763,326.597359,0.000000,0
1,17.777371,5.022056,0.405121,0.405121,1.335460,0.676128,1.000000,0.073301,61.828454,134.429902,...,0.455641,0.500191,1.125059,0.804943,0.910933,0.008382,50.308317,276.608621,366.879367,0
2,11.855231,6.372713,2.939412,2.939412,50.054526,2.089913,1.000000,0.192623,188.645150,29.310724,...,1.585416,1.585427,378.147222,1.050186,0.999993,0.427538,283.190442,195.587902,89.918697,0
3,13.911165,16.481641,5.661395,5.661360,13.304319,6.360569,1.000006,0.101952,357.850150,239.631257,...,5.801234,5.801234,12.686171,6.682304,1.000000,0.090600,271.614982,288.643750,0.000000,0
4,14.933960,24.877081,6.295984,6.295984,0.855904,5.916377,1.000000,-0.079627,232.387033,173.791784,...,7.068596,7.068596,0.731521,7.849565,1.000000,-0.118476,241.826172,183.835760,57.286862,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,16.007338,17.450000,1.050000,1.050000,1.240000,0.840000,1.000000,0.101211,154.000000,254.000000,...,1.060000,1.060000,1.150000,0.810000,1.000000,0.070620,139.000000,184.000000,182.000000,1
141,15.875201,28.230000,2.020000,2.020000,1.770000,0.600000,1.000000,0.273218,91.000000,80.000000,...,1.590000,1.590000,1.720000,0.540000,1.000000,0.272412,90.000000,87.000000,158.000000,1
142,14.378253,68.390000,2.810000,2.840000,6.520000,1.690000,0.990000,0.308747,112.000000,106.000000,...,1.930000,4.020000,13.980000,3.700000,0.480000,0.371691,131.000000,250.000000,550.000000,1
143,14.597912,12.920000,1.010000,1.010000,0.730000,0.830000,1.000000,-0.203044,38.000000,70.000000,...,1.130000,1.130000,0.970000,0.380000,1.000000,-0.025060,62.000000,50.000000,42.000000,1


In [3]:
df.columns

Index(['SIZE-3year', 'GR-PROFIT-MARGIN-3year',
       'CAPITAL-EMPLOYED-TURNOVER-3year', 'STOCKHOLDERS-EQUITY-TURNOVER-3year',
       'CAPITAL-EMPLOYED/NET-FIXED-ASSETS-3year', 'DEBT/EQUITY-3year',
       'EQUITY/CAPITAL-EMPLOYED-3year', 'WORKING-CAPITAL/TOTAL-ASSETS-3year',
       'AV-COLLECTION-PERIOD-FOR-RECIEVABLES-3year', 'AV-PAYMENT-PERIOD-3year',
       'AV-TURNOVER-PERIOD-FOR-INVENTORIES-3year',
       'SCALED-CHANGE-ASSETS-2year', 'SIZE-2year', 'CH.NET-INCOME-2year',
       'GR-PROFIT-MARGIN-2year', 'CAPITAL-EMPLOYED-TURNOVER-2year',
       'STOCKHOLDERS-EQUITY-TURNOVER-2year',
       'CAPITAL-EMPLOYED/NET-FIXED-ASSETS-2year', 'DEBT/EQUITY-2year',
       'EQUITY/CAPITAL-EMPLOYED-2year', 'WORKING-CAPITAL/TOTAL-ASSETS-2year',
       'AV-COLLECTION-PERIOD-FOR-RECIEVABLES-2year', 'AV-PAYMENT-PERIOD-2year',
       'AV-TURNOVER-PERIOD-FOR-INVENTORIES-2year', 'final'],
      dtype='object')

## Splitting into input(X), target(Y), train(80%) , test(20%) and normalizing the data

In [4]:
np.random.seed(333)

X=df.iloc[:,0:-1].values
Y=df.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=43) 
X_train=preprocessing.normalize(X_train)
X_test=preprocessing.normalize(X_test)
y_train=y_train.astype(int)
y_test=y_test.astype(int)

## Cart Decision Tree Algorithm


In [5]:
X_cart_train=X_train
X_cart_test=X_test
y_cart_train=y_train
y_cart_test=y_test

In [6]:
np.random.seed(112)

clf = DecisionTreeClassifier(criterion="gini")
clf=clf.fit(X_cart_train,y_cart_train)
depth=clf.get_depth()

parameters={'criterion':('gini','entropy'),'max_depth': range(1,depth,1),'max_features':('sqrt','log2')}
clf_tree=DecisionTreeClassifier()
clf=GridSearchCV(clf_tree,parameters)
clf=clf.fit(X_cart_train,y_cart_train)
y_cart_pred = clf.predict(X_cart_test)
auc_cart=metrics.roc_auc_score(y_cart_test,y_cart_pred)

print(auc_cart)
print(clf.best_estimator_)


0.65
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
                       max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


## Naive Bayes

In [7]:
X_nb_train=X_train
X_nb_test=X_test
y_nb_train=y_train
y_nb_test=y_test

In [8]:
gnb = GaussianNB(var_smoothing=1e-09)
y_nb_pred = gnb.fit(X_nb_train,y_nb_train).predict(X_nb_test)

auc_nb=metrics.roc_auc_score(y_nb_test,y_nb_pred)

print(auc_nb)

0.5710526315789474


## Logistic Regression

In [9]:
X_lr_train=X_train
X_lr_test=X_test
y_lr_train=y_train
y_lr_test=y_test

In [10]:
parameters={'penalty':('l2','none'),'class_weight':(None,'balanced'),'solver':('newton-cg','lbfgs','sag')}

logreg = LogisticRegression()
log=GridSearchCV(logreg,parameters)

log.fit(X_lr_train, y_lr_train)

y_lr_pred = log.predict(X_lr_test)

auc_lr=metrics.roc_auc_score(y_lr_test,y_lr_pred)

print(auc_lr)
print(log.best_estimator_)

0.5
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)


## Dataset "2 years before"

In [11]:
df2=pd.read_csv("2 years before.csv")
df2['final'][df2['final']=='Bankrupted']=0
df2['final'][df2['final']=='Non-Bankrupted']=1
df2.head()

bankruptcies=df2['final'][df2['final']==0].count()
rows=df2.shape[0]
columns=df2.shape[1]
print(f'Our dataset is comprised of {rows} rows and {columns} columns. We have {bankruptcies} bankrupted companies and {rows-bankruptcies} companies that still operate.')

Our dataset is comprised of 145 rows and 12 columns. We have 49 bankrupted companies and 96 companies that still operate.


## Splitting into input(X), target(Y), train(80%) , test(20%) and normalizing the data

In [12]:
np.random.seed(333)

X=df2.iloc[:,0:-1].values
Y=df2.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=43) 
X_train=preprocessing.normalize(X_train)
X_test=preprocessing.normalize(X_test)
y_train=y_train.astype(int)
y_test=y_test.astype(int)

## Cart Decision Tree Algorithm


In [13]:
X_cart_train=X_train
X_cart_test=X_test
y_cart_train=y_train
y_cart_test=y_test

In [14]:
np.random.seed(11)

clf2 = DecisionTreeClassifier(criterion="gini")
clf2=clf2.fit(X_cart_train,y_cart_train)
depth=clf2.get_depth()

parameters={'criterion':('gini','entropy'),'max_depth': range(1,depth,1),'max_features':('sqrt','log2')}
clf_tree2=DecisionTreeClassifier()
clf2=GridSearchCV(clf_tree2,parameters)
clf2=clf2.fit(X_cart_train,y_cart_train)
y_cart_pred = clf2.predict(X_cart_test)
auc_cart2=metrics.roc_auc_score(y_cart_test,y_cart_pred)

print(auc_cart2)
print(clf2.best_estimator_)


0.6210526315789473
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
                       max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


## Naive Bayes

In [15]:
X_nb_train=X_train
X_nb_test=X_test
y_nb_train=y_train
y_nb_test=y_test

In [16]:
gnb2= GaussianNB(var_smoothing=1e-09)
y_nb_pred = gnb2.fit(X_nb_train,y_nb_train).predict(X_nb_test)

auc_nb2=metrics.roc_auc_score(y_nb_test,y_nb_pred)

print(auc_nb2)

0.5473684210526315


## Logistic Regression

In [17]:
X_lr_train=X_train
X_lr_test=X_test
y_lr_train=y_train
y_lr_test=y_test

In [18]:
parameters={'penalty':('l2','none'),'class_weight':(None,'balanced'),'solver':('newton-cg','lbfgs','sag')}

logreg2 = LogisticRegression()
log2=GridSearchCV(logreg2,parameters)

log2.fit(X_lr_train, y_lr_train)

y_lr_pred = log2.predict(X_lr_test)

auc_lr2=metrics.roc_auc_score(y_lr_test,y_lr_pred)

print(auc_lr2)
print(log2.best_estimator_)

0.5236842105263158
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)


## Dataset "last year"

In [19]:
df3=pd.read_csv("last year.csv")
df3['final'][df3['final']=='Bankrupted']=0
df3['final'][df3['final']=='Non-Bankrupted']=1
df3.head()

bankruptcies=df3['final'][df3['final']==0].count()
rows=df3.shape[0]
columns=df3.shape[1]
print(f'Our dataset is comprised of {rows} rows and {columns} columns. We have {bankruptcies} bankrupted companies and {rows-bankruptcies} companies that still operate.')

Our dataset is comprised of 145 rows and 38 columns. We have 49 bankrupted companies and 96 companies that still operate.


## Splitting into input(X), target(Y), train(80%) , test(20%) and normalizing the data

In [20]:
np.random.seed(333)

X=df3.iloc[:,0:-1].values
Y=df3.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=43) 
X_train=preprocessing.normalize(X_train)
X_test=preprocessing.normalize(X_test)
y_train=y_train.astype(int)
y_test=y_test.astype(int)

## Cart Decision Tree Algorithm


In [21]:
X_cart_train=X_train
X_cart_test=X_test
y_cart_train=y_train
y_cart_test=y_test

In [22]:
np.random.seed(33)

clf3 = DecisionTreeClassifier(criterion="gini")
clf3=clf3.fit(X_cart_train,y_cart_train)
depth=clf3.get_depth()

parameters={'criterion':('gini','entropy'),'max_depth': range(1,depth,1),'max_features':('sqrt','log2')}
clf_tree3=DecisionTreeClassifier()
clf3=GridSearchCV(clf_tree3,parameters)
clf3=clf3.fit(X_cart_train,y_cart_train)
y_cart_pred = clf3.predict(X_cart_test)
auc_cart3=metrics.roc_auc_score(y_cart_test,y_cart_pred)

print(auc_cart3)
print(clf3.best_estimator_)


0.818421052631579
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')


In [51]:

xxxx=DecisionTreeClassifier(criterion='gini',max_depth=6,max_features='log2')
xxxx=xxxx.fit(X_cart_train,y_cart_train)
pred=xxxx.predict(X_cart_test)
cc=metrics.roc_auc_score(y_cart_test,pred)
print(cc,df3.columns)
xxxx.feature_importances_

0.618421052631579 Index(['SIZE-3year', 'GR-PROFIT-MARGIN-3year',
       'CAPITAL-EMPLOYED-TURNOVER-3year', 'STOCKHOLDERS-EQUITY-TURNOVER-3year',
       'CAPITAL-EMPLOYED/NET-FIXED-ASSETS-3year', 'DEBT/EQUITY-3year',
       'EQUITY/CAPITAL-EMPLOYED-3year', 'WORKING-CAPITAL/TOTAL-ASSETS-3year',
       'AV-COLLECTION-PERIOD-FOR-RECIEVABLES-3year', 'AV-PAYMENT-PERIOD-3year',
       'AV-TURNOVER-PERIOD-FOR-INVENTORIES-3year',
       'SCALED-CHANGE-ASSETS-2year', 'SIZE-2year', 'CH.NET-INCOME-2year',
       'GR-PROFIT-MARGIN-2year', 'CAPITAL-EMPLOYED-TURNOVER-2year',
       'STOCKHOLDERS-EQUITY-TURNOVER-2year',
       'CAPITAL-EMPLOYED/NET-FIXED-ASSETS-2year', 'DEBT/EQUITY-2year',
       'EQUITY/CAPITAL-EMPLOYED-2year', 'WORKING-CAPITAL/TOTAL-ASSETS-2year',
       'AV-COLLECTION-PERIOD-FOR-RECIEVABLES-2year', 'AV-PAYMENT-PERIOD-2year',
       'AV-TURNOVER-PERIOD-FOR-INVENTORIES-2year', 'SCALED-CHANGE-ASSETS',
       'SIZE', 'CH.NET-INCOME', 'GR-PROFIT-MARGIN',
       'CAPITAL-EMPLOYED-TURNOVE

array([0.05997821, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.06343849, 0.12049194, 0.11346927, 0.05416782, 0.        ,
       0.04123502, 0.        , 0.08574794, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02474101,
       0.        , 0.        , 0.07737448, 0.        , 0.        ,
       0.01955811, 0.        , 0.13988187, 0.19991584, 0.        ,
       0.        , 0.        ])

## Naive Bayes

In [147]:
X_nb_train=X_train
X_nb_test=X_test
y_nb_train=y_train
y_nb_test=y_test

In [148]:
gnb3= GaussianNB(var_smoothing=1e-09)
y_nb_pred = gnb3.fit(X_nb_train,y_nb_train).predict(X_nb_test)

auc_nb3=metrics.roc_auc_score(y_nb_test,y_nb_pred)

print(auc_nb3)

0.5710526315789474


## Logistic Regression

In [149]:
X_lr_train=X_train
X_lr_test=X_test
y_lr_train=y_train
y_lr_test=y_test

In [150]:
parameters={'penalty':('l2','none'),'class_weight':(None,'balanced'),'solver':('newton-cg','lbfgs','sag')}

logreg3 = LogisticRegression()
log3=GridSearchCV(logreg3,parameters)

log3.fit(X_lr_train, y_lr_train)

y_lr_pred = log3.predict(X_lr_test)

auc_lr3=metrics.roc_auc_score(y_lr_test,y_lr_pred)

print(auc_lr3)
print(log3.best_estimator_)

0.5
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)


## All the results

In [164]:
data={'Logistic_Reg':[auc_lr,auc_lr2,auc_lr3],
    'Naive_Bayes':[auc_nb,auc_nb2,auc_nb3],
     'CART':[auc_cart,auc_cart2,auc_cart3]}

results=pd.DataFrame(data,index=['1 years before','2 years before','last year'])

results

,Logistic_Reg,Naive_Bayes,CART
1 years before,0.500000,0.571053,0.650000
2 years before,0.523684,0.547368,0.621053
last year,0.500000,0.571053,0.818421
